In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

flight_data = pd.read_csv("C:/Users/Emily Shao/Desktop/archive/itineraries.csv")

In [9]:
# MAY NOT BE NECESSARY
# THIS SECTION MAY NOT BE NECESSARY, TRYING TO CREATE CATEGORICAL DATA
# DO NOT RUN THIS BLOCK

from datetime import datetime 

type(flight_data['flightDate'][0])

flight_data['flightDate'][1048500] # '4/17/2022' type: str

def time_buckets(pdseries):
    for date in pdseries:
        date_object = datetime.strptime(date, '%m/%d/%Y')


def str_to_date(str):
    return datetime.strptime(str, '%m/%d/%Y')


flight_data['date_in_time'] = flight_data['flightDate'].apply(str_to_date)


In [11]:
flight_data['date_in_time']
type(flight_data['date_in_time'][0])
flight_data['date_in_time']

0         2022-04-17
1         2022-04-17
2         2022-04-17
3         2022-04-17
4         2022-04-17
             ...    
1048570   2022-06-05
1048571   2022-06-05
1048572   2022-06-05
1048573   2022-06-05
1048574   2022-06-05
Name: date_in_time, Length: 1048575, dtype: datetime64[ns]

In [12]:
import numpy as np

type(flight_data['baseFare'][0])

print(flight_data['baseFare'].max())

def data_buckets(pdseries, num_buckets):
    bins = np.linspace(0, 4000, num_buckets + 1)
    labels = np.ceil(bins[1:]).astype(int)

    bucket_labels = pd.cut(pdseries, bins=bins, labels=labels, right=True)

    return bucket_labels



buckets = 25 #100 # 25 400
flight_data['price_buckets'] = data_buckets(flight_data['baseFare'], buckets)




# # Checking if there are NaN values within the code
# print("Column 'A' has NaN values:", flight_data['price_buckets'].isna().any())

# nan_rows = flight_data[flight_data['price_buckets'].isna()]
# print(nan_rows[['price_buckets', 'baseFare']])




# def to_str(input):
#     return str(input)

# flight_data['price_categories'] = flight_data['price_buckets'].apply(to_str)

# print(flight_data['baseFare'],flight_data['price_buckets']) #, flight_data['price_categories'])


3531.16


In [13]:
encoder = OneHotEncoder(sparse=False)
# encoded_features = encoder.fit_transform(flight_data[['startingAirport', 'destinationAirport', 'flightDate']])
encoded_features = encoder.fit_transform(flight_data[['startingAirport', 'destinationAirport', 'date_in_time']])

# X = pd.concat([pd.DataFrame(encoded_features), flight_data[['baseFare']]], axis=1)

X = pd.concat([pd.DataFrame(encoded_features), flight_data[['price_buckets']]], axis=1)
y = X.pop('price_buckets')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

c:\Users\Emily Shao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
# output trained with 25 buckets and 1000 iterations = accuracy 0.48353
# output trained with 25 buckets and 300 iterations = accuracy 0.48344
# output trained with 25 buckets and 100 iterations = accuracy 0.48285

# output trained with 100 buckets and 300 iterations = accuracy 0.4833, time = 4m 42sec, iterations limit reached
# output trained with 100 buckets and 500 iterations = accuracy 0.4834, time = 10m 4sec, failed to converge

# output trained with 25 buckets and 500 iterations (with DateTime) = accuracy 0.4834, time = 9m 18sec, iterations limit reached


model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4834608873947977


In [8]:
type(flight_data['flightDate'][0])

str

In [ ]:
model = LogisticRegression(max_iter=100)

# Initialize accuracy variable to keep track of progress
accuracy_progress = []

for i in range(1, 101):  # Train for 100 iterations
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_progress.append(accuracy)
    print("Iteration {}: Accuracy: {:.2f}%".format(i, accuracy * 100))

# Final accuracy after training for 100 iterations
final_accuracy = accuracy_progress[-1]
print("Final Accuracy:", final_accuracy)

In [ ]:
num_epochs = 100

# Create lists to store training accuracy for each epoch
# train_accuracy_list = []

# Training loop
for epoch in range(1, num_epochs + 1):
    # Train the model for one epoch
    model.fit(X_train, y_train)
    
    # Evaluate the model on the training set
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    # train_accuracy_list.append(train_accuracy)
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    # Print progress
    print(f"Epoch {epoch}/{num_epochs} - Training Accuracy: {train_accuracy:.4f} - Test Accuracy: {test_accuracy:.4f}")

# After training, print the final accuracy
final_test_accuracy = accuracy_score(y_test, y_pred)
print("Final Test Accuracy:", final_test_accuracy)

In [ ]:
# TODO

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(feature_names, model.coef_[0])
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Coefficients of Logistic Regression Model')
plt.show()